# Generate Random Object Sorting Tasks

In [2]:
import numpy as np

In [2]:
def create_sorting_dataset(objects, seqs_length, n_seqs):

    n_objects = len(objects)

    # generate random permutations of length `seqs_length` out of `vocab_size`
    seqs = np.array([np.random.choice(range(n_objects), size=seqs_length, replace=False) for _ in range(n_seqs)])
    
    # remove duplicate seqs (although very unlikely)
    _, unique_seq_idxs = np.unique(seqs, axis=0, return_inverse=True)
    seqs = seqs[unique_seq_idxs]

    # create object sequences
    object_seqs = objects[seqs]
    
    sorted_seqs = np.sort(seqs, axis=1)

    arg_sort = np.argsort(seqs, axis=1)

    
    # add `START_TOKEN` to beginning of sorting 
    start_token = seqs_length
    start_tokens = np.array([start_token] * len(arg_sort))[np.newaxis].T
    arg_sort = np.hstack([start_tokens, arg_sort])

    return seqs, sorted_seqs, arg_sort, object_seqs, start_token

## Task 1

generate `vocab_size` objects as random gaussian vectors with dimension `dim`. Associate a random ordering to them.

In [3]:
# dataset 1
vocab_size = 64
dim = 8
seqs_length = 10
n_seqs = 10_0000

objects = np.random.normal(size=(vocab_size, dim))

seqs, sorted_seqs, arg_sort, object_seqs, start_token = create_sorting_dataset(objects, seqs_length, n_seqs)

target = arg_sort[:, :-1]
labels = arg_sort[:, 1:]

data = {
    'objects': objects, 'seqs': seqs, 'sorted_seqs': sorted_seqs, 'arg_sort': arg_sort,
    'object_seqs': object_seqs, 'target': target, 'labels': labels, 'start_token': start_token
    }

np.save('object_sorting_datasets/task1_object_sort_dataset.npy', data)

## Task 2

independently generate another random set of objects and associated ordering.

In [4]:
# dataset 2 (same paramters, just re-generate objects randomly)
vocab_size = 64
dim = 8
seqs_length = 10
n_seqs = 10_0000

objects = np.random.normal(size=(vocab_size, dim))

seqs, sorted_seqs, arg_sort, object_seqs, start_token = create_sorting_dataset(objects, seqs_length, n_seqs)

target = arg_sort[:, :-1]
labels = arg_sort[:, 1:]

data = {
    'objects': objects, 'seqs': seqs, 'sorted_seqs': sorted_seqs, 'arg_sort': arg_sort,
    'object_seqs': object_seqs, 'target': target, 'labels': labels, 'start_token': start_token
    }

np.save('object_sorting_datasets/task2_object_sort_dataset.npy', data)

## Task 2 Reshuffled

Reshuffle the order of the objects in task 2

In [5]:
data = np.load('object_sorting_datasets/task2_object_sort_dataset.npy', allow_pickle=True).item()
objects = data['objects']
seqs = data['seqs']

reshuffle = np.random.choice(64, size=64, replace=False)
objects_ = objects[reshuffle]
object_seqs_ = objects_[seqs]

data['reshuffle'] = reshuffle
data['objects'] = objects_
data['object_seqs'] = object_seqs_

np.save('object_sorting_datasets/task2_reshuffled_object_sort_dataset.npy', data)

## Generate objects with attribute-product structure

Generate two attributes as random gaussian vectors and associate an ordering to them. Then generate objects as cartesian products. Associate an ordering to them where one attribute forms a primary key, and the other forms a secondary key.

In [6]:
# generate objects with attribute-product structure

attr1_n_objects = 4 # number of possible values for attribute 1
attr1_embedding_dim = 4 # dimension of vector representation of attribute 1 
attr2_n_objects = 12 # number of possible values for attribute 2
attr2_embedding_dim = 8 # dimension of vector representation of attribute 2

# generate vector representations of the two attributes
attr1_objects = np.random.normal(size=(attr1_n_objects,  attr1_embedding_dim))
attr2_objects = np.random.normal(size=(attr2_n_objects,  attr2_embedding_dim))

# generate attribute-product objects and ordering 
object_products = [(attr1, attr2) for attr1 in range(attr1_n_objects) for attr2 in range(attr2_n_objects)]

objects = []
for attr1, attr2 in object_products:
    attr1_object = attr1_objects[attr1] # get vector representation of attribute 1
    attr2_object = attr2_objects[attr2] # get vector representation of attribute 2
    object_ = np.concatenate([attr1_object, attr2_object]) # stack attributes to create object
    objects.append(object_)

objects = np.array(objects)

In [7]:
objects.shape # (n_objects, object_dim)

(48, 12)

In [8]:
# generate object sorting dataset
seqs_length = 10
n_seqs = 10_0000

seqs, sorted_seqs, arg_sort, object_seqs, start_token = create_sorting_dataset(objects, seqs_length, n_seqs)

target = arg_sort[:, :-1]
labels = arg_sort[:, 1:]

data = {
    'objects': objects, 'attr1_objects': attr1_objects, 'attr2_objects': attr2_objects, 
    'seqs': seqs, 'sorted_seqs': sorted_seqs, 'arg_sort': arg_sort,
    'object_seqs': object_seqs, 'target': target, 'labels': labels, 'start_token': start_token
    }

np.save('object_sorting_datasets/product_structure_object_sort_dataset.npy', data)

## Reshuffle Attribute-Product Structure Dataset

In [9]:
attr1_reshuffle = np.random.choice(attr1_n_objects, size=attr1_n_objects, replace=False)
attr2_reshuffle = np.arange(attr2_n_objects) # identity permutation

attr1_objects_reshuffled = attr1_objects[attr1_reshuffle]
attr2_objects_reshuffled = attr2_objects[attr2_reshuffle]

# generate attribute-product objects and ordering 
object_products = [(attr1, attr2) for attr1 in range(attr1_n_objects) for attr2 in range(attr2_n_objects)]

objects_reshuffled = []
for attr1, attr2 in object_products:
    attr1_object = attr1_objects[attr1] # get vector representation of attribute 1
    attr2_object = attr2_objects[attr2] # get vector representation of attribute 2
    object_ = np.concatenate([attr1_object, attr2_object]) # stack attributes to create object
    objects_reshuffled.append(object_)

objects_reshuffled = np.array(objects)


In [ ]:

eval_task_data = np.load('object_sorting_datasets/product_structure_object_sort_dataset.npy', allow_pickle=True).item()

objects, seqs, sorted_seqs, object_seqs, target, labels, start_token = (eval_task_data['objects'], eval_task_data['seqs'], eval_task_data['sorted_seqs'], eval_task_data['object_seqs'], \
    eval_task_data['target'], eval_task_data['labels'], eval_task_data['start_token'])
print(object_seqs.shape, target.shape, labels.shape)
import sys; sys.path.append('../'); sys.path.append('../..')

import autoregressive_abstractor
import tensorflow as tf
from transformer_modules import TeacherForcingAccuracy
teacher_forcing_acc_metric = TeacherForcingAccuracy(ignore_class=None)
metrics = [teacher_forcing_acc_metric]
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, ignore_class=None, name='sparse_categorical_crossentropy')
create_opt = lambda : tf.keras.optimizers.Adam()

seqs_length = 30
autoreg_abstractor_kwargs = dict(
        encoder_kwargs=dict(num_layers=2, num_heads=4, dff=64, dropout_rate=0.1),
        abstractor_kwargs=dict(
            num_layers=2,
            dff=64,
            rel_dim=4,
            symbol_dim=64,
            proj_dim=8,
            symmetric_rels=False,
            encoder_kwargs=dict(use_bias=True),
            rel_activation_type='softmax',
            use_self_attn=False,
            use_layer_norm=False,
            dropout_rate=0.2),
        decoder_kwargs=dict(num_layers=1, num_heads=4, dff=64, dropout_rate=0.1),
        input_vocab='vector',
        target_vocab=seqs_length+1,
        embedding_dim=64,
        output_dim=seqs_length,
        abstractor_type='abstractor',
        abstractor_on='encoder',
        decoder_on='abstractor',
        name='autoregressive_abstractor')


argsort_model = autoregressive_abstractor.AutoregressiveAbstractor(
                **autoreg_abstractor_kwargs)
argsort_model.compile(loss=loss, optimizer=create_opt(), metrics=metrics)
x = argsort_model((object_seqs[:32], target[:32]));
print(x.shape)
print(argsort_model.summary())


(100000, 10, 12) (100000, 10) (100000, 10)
(32, 10, 30)
Model: "autoregressive_abstractor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 source_embedder (Dense)     multiple                  832       
                                                                 
 target_embedder (Embedding)  multiple                 1984      
                                                                 
 add_pos_embedding_input (Ad  multiple                 0         
 dPositionalEmbedding)                                           
                                                                 
 add_pos_embedding_target (A  multiple                 0         
 ddPositionalEmbedding)                                          
                                                                 
 Encoder_General (Encoder)   multiple                  149888    
                                                   

In [10]:
data['attr1_reshuffle'] = attr1_reshuffle
data['attr2_reshuffle'] = attr2_reshuffle
data['objects'] = objects_reshuffled

object_seqs_reshuffled = objects_reshuffled[seqs]
data['object_seqs'] = object_seqs_reshuffled

data['attr1_objects'] = attr1_objects_reshuffled
data['attr2_objects'] = attr2_objects_reshuffled

np.save('object_sorting_datasets/product_structure_reshuffled_object_sort_dataset.npy', data)